<a href="https://colab.research.google.com/github/Picklin/PAE/blob/main/Robot_Music%C3%B3logo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PAE - BMAT

In [ ]:
!pip install openai>=1.14 gradio --quiet

In [ ]:
# 2. Cargar la clave desde los secretos
from google.colab import userdata
import os
KEY = userdata.get("OPENAI_API_KEY")


In [ ]:
#Versión de la presentación del prototipo
from openai import OpenAI
import os

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= KEY
)
SYSTEM_PROMPT = "Eres un experto en música. Devuelve el nombre del artista y el nombre de la canción del siguiente título de vídeo de YouTube."

def musicologo(titulo_video):
    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"Título del vídeo: {titulo_video}"}
            ],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"⚠️ Error: {str(e)}"


In [ ]:
import gradio as gr

iface = gr.Interface(fn=musicologo,
                     inputs=gr.Textbox(label="Título del vídeo de YouTube"),
                     outputs=gr.Textbox(label="output"),
                     title="🎧 Robot Musicólogo con Deepseek")
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6229e7892848a7e69e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#Versión para la presentación final
#Versión de la presentación del prototipo
'''from openai import OpenAI
import os

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= KEY
)
SYSTEM_PROMPT = "Eres un experto en música. Devuelve el nombre del artista y el nombre de la canción del siguiente título de vídeo de YouTube."

def procesar_entradas(text_input, file):
    titulos = []

    # Si se ha escrito texto
    if text_input:
        titulos += [line.strip() for line in text_input.split("\n") if line.strip()]

    # Si se ha subido un archivo
    if file is not None:
        with open(file.name, 'r', encoding='utf-8') as f:
            titulos += [line.strip() for line in f if line.strip()]

    # Evitar continuar si no hay títulos
    if not titulos:
        return "No se han proporcionado títulos para analizar."

    resultados = []

    for titulo in titulos:
        try:
            response = client.chat.completions.create(
                model="deepseek/deepseek-r1:free",
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Título del vídeo: {titulo}"}
                ],
                temperature=0
            )
            respuesta = response.choices[0].message.content.strip()
            resultados.append(f"Título original: {titulo}\n{respuesta}\n")
        except Exception as e:
            return f"⚠️ Error: {str(e)}"

    output_file = "/content/resultados.txt"
    with open(output_file, "w", encoding="utf-8") as out:
        out.write("\n".join(resultados))

    return output_file


In [ ]:
#Versión mejorada con ejecución en paralelo
'''from openai import OpenAI
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= KEY
)
SYSTEM_PROMPT = "Eres un experto en música. Devuelve el nombre del artista y el nombre de la canción del siguiente título de vídeo de YouTube."

def procesar_titulo(titulo):
  try:
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1:free",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Título del vídeo: {titulo}"}
        ],
        temperature=0
    )
    respuesta = response.choices[0].message.content.strip()
  except Exception as e:
    return f"⚠️ Error: {str(e)}"
  return f"Título original: {titulo}\n{respuesta}\n"

def procesar_entradas(text_input, file):
  titulos = []

  if text_input:
      titulos += [line.strip() for line in text_input.split("\n") if line.strip()]

  if file is not None:
      with open(file.name, 'r', encoding='utf-8') as f:
          titulos += [line.strip() for line in f if line.strip()]

  if not titulos:
      return "No se han proporcionado títulos para analizar."

  resultados = []
  with ThreadPoolExecutor(max_workers=5) as executor:
      futuros = [executor.submit(procesar_titulo, titulo) for titulo in titulos]
      for future in as_completed(futuros):
          resultados.append(future.result())

  output_path = "/content/resultados.txt"
  with open(output_path, "w", encoding="utf-8") as f:
      f.write("\n".join(resultados))

  return output_path


In [ ]:
#versión definitiva de un único prompt
from openai import OpenAI
import os

# Inicialización del cliente OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=KEY  # Asegúrate de que KEY esté definido correctamente
)

# Prompt general para todo el listado
SYSTEM_PROMPT = (
    "Eres un experto en música. A continuación se muestra una lista de títulos de vídeos de YouTube. "
    "Extrae para cada uno el nombre del artista y el nombre de la canción, usando este formato:\n\n"
    "Título: <título original>\nArtista: <nombre del artista>\nCanción: <nombre de la canción>\n\n"
)

def procesar_listado_completo(text_input, file):
    titulos = []

    if text_input:
        titulos += [line.strip() for line in text_input.split("\n") if line.strip()]

    if file is not None:
        with open(file.name, 'r', encoding='utf-8') as f:
            titulos += [line.strip() for line in f if line.strip()]

    if not titulos:
        return "No se han proporcionado títulos para analizar."

    # Construcción del bloque de entrada
    listado = "\n".join(f"{i+1}. {t}" for i, t in enumerate(titulos))
    user_prompt = f"Lista de títulos:\n{listado}"

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1:free",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=2048  # ajusta si hace falta
        )
        resultado = response.choices[0].message.content.strip()
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

    # Guardar en archivo
    output_path = "/content/resultados.txt"
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(resultado)

    return output_path


In [ ]:
import gradio as gr

iface = gr.Interface(fn=procesar_listado_completo,
                     inputs=[
                         gr.Textbox(label="Introduce títulos manualmente (uno por línea)", lines=8, placeholder="Ej: Adele - Hello"),
                         gr.File(label="O sube un archivo .txt con los títulos (uno por línea)", file_types=[".txt"]),
                     ],
                     outputs=[
                         gr.File(label="Descargar resultados"),
                     ],
                     title="🎧 Robot Musicólogo")
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a95cd39c600e6deaba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
